In [1]:
! pip install datasets
! pip install transformers==4.45.2 sentence-transformers==3.1.1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from transformers import AutoModelForSequenceClassification,AutoModelForCausalLM, AutoTokenizer,TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import torch
# 加载 Llama2-7B
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"  # 适用于 GPT 类模型
)
model = get_peft_model(model, lora_config)  # 应用 LoRA
tokenizer = AutoTokenizer.from_pretrained(model_name)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [5]:
# 配置 LoRA 适配层
# 应用 LoRA
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/mapping.py:185: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [6]:
device = model.device

In [7]:
! ls

drive  logs  results  sample_data  wandb


In [8]:


from sklearn.metrics import accuracy_score, classification_report
# 3️⃣ 处理数据
def preprocess_function(examples):
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    inputs["labels"] = [float(label) for label in examples["label"]]  # 确保 label 是 float
    return inputs

dataset = load_dataset("json", data_files={"train": "drive/MyDrive/Colab/llm_demo/train.json", "test": "drive/MyDrive/Colab/llm_demo/train.json"})
tokenized_datasets = dataset.map(preprocess_function, batched=True)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# 4️⃣ 训练参数
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    logging_dir="./logs",
    learning_rate=2e-5
)



# 6️⃣ 训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)



Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: haoyuh (haoyuh-university-of-michigan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.580140,0.500000
2,No log,1.471590,0.500000
3,No log,1.384634,0.500000
4,No log,1.331336,0.500000
5,No log,1.308993,0.500000


TrainOutput(global_step=65, training_loss=1.4566213754507211, metrics={'train_runtime': 542.2231, 'train_samples_per_second': 0.24, 'train_steps_per_second': 0.12, 'total_flos': 16380283944960.0, 'train_loss': 1.4566213754507211, 'epoch': 5.0})

In [10]:
import torch
import torch.nn.functional as F

def predict(text):
    # 确保模型在正确的设备上
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 预处理输入文本
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=256).to(device)

    # 计算 logits（模型原始输出）
    with torch.no_grad():
        logits = model(**inputs).logits  # logits 形状: (1, 1)
        prob = torch.sigmoid(logits)  # 计算概率（0~1）
    probs = torch.softmax(logits, dim=-1)  # 使用 softmax 获取概率
    class_1_probs = probs[:, 1].numpy()    # 获取类别 1 的概率

    return class_1_probs  # 返回单个数值




In [13]:
# **测试预测**
text1 = "用户过去一年内信用评分保持稳定，无不良记录。"
text2 = "用户近期信用卡账单连续三个月未全额还款，负债率较高。"
text3 = "用户最近六个月内按时还款，无逾期记录。"

print(f"Text1 逾期概率: {predict(text1)[0]:.4f}")
print(f"Text2 逾期概率: {predict(text2)[0]:.4f}")
print(f"Text3 逾期概率: {predict(text3)[0]:.4f}")

Text1 逾期概率: 0.8737
Text2 逾期概率: 0.9268
Text3 逾期概率: 0.9017


In [12]:
prob1[0]

NameError: name 'prob1' is not defined